今天用的是信贷预测数据集

In [1]:
import pandas as pd
 
data=pd.read_csv('data.csv')

data.head()

,Id,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


# 数据预处理
对数据进行编码和缺失值填补

In [2]:
import pandas as pd    
import numpy as np     
import matplotlib.pyplot as plt    
import seaborn as sns   
import warnings
warnings.filterwarnings('ignore') 
 

plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False    

# 先筛选字符串变量 
discrete_features = data.select_dtypes(include=['object']).columns.tolist()
# Home Ownership 标签编码
maps={
    'Home Ownership' : {
    'Own Home': 1,
    'Rent': 2,
    'Have Mortgage': 3,
    'Home Mortgage': 4
    },
    'Years in current job':{
    '< 1 year': 1,
    '1 year': 2,
    '2 years': 3,
    '3 years': 4,
    '4 years': 5,
    '5 years': 6,
    '6 years': 7,
    '7 years': 8,
    '8 years': 9,
    '9 years': 10,
    '10+ years': 11
},
    'Term':{
    'Short Term': 0,
    'Long Term': 1
} 
}

data=data.replace(maps)

# Purpose 独热编码，记得需要将bool类型转换为数值
data=pd.get_dummies(data,columns=['Purpose'])
data2=pd.read_csv('data.csv')
list_diff=data.columns.difference(data2.columns)
data[list_diff]=data[list_diff].astype(int)

data.rename(columns={'Term': 'Long Term'}, inplace=True) # 重命名列
continuous_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()  #把筛选出来的列名转换成列表
 
 # 连续特征用中位数补全
for feature in continuous_features:     
    mode_value = data[feature].mode()[0]            #获取该列的众数。
    data[feature].fillna(mode_value, inplace=True)          #用众数填充该列的缺失值，inplace=True表示直接在原数据上修改。

data.head()

,Id,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,...,Purpose_home improvements,Purpose_major purchase,Purpose_medical bills,Purpose_moving,Purpose_other,Purpose_renewable energy,Purpose_small business,Purpose_take a trip,Purpose_vacation,Purpose_wedding
0,0,1,482087.0,11.0,0.0,11.0,26.3,685960.0,1.0,14.0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1025487.0,11.0,0.0,15.0,15.3,1181730.0,0.0,14.0,...,0,0,0,0,0,0,0,0,0,0
2,2,4,751412.0,9.0,0.0,11.0,35.0,1182434.0,0.0,14.0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,805068.0,7.0,0.0,8.0,22.5,147400.0,1.0,14.0,...,0,0,0,0,0,0,0,0,0,0
4,4,2,776264.0,9.0,0.0,13.0,13.6,385836.0,1.0,14.0,...,0,0,0,0,0,0,0,0,0,0


---
# 数据集划分
## 方案一：三分法（训练集 + 验证集 + 测试集）

当不使用交叉验证时，需要划分出验证集用于调参。

In [3]:
from sklearn.model_selection import train_test_split

X=data.drop(['Credit Default'],axis=1)
y=data['Credit Default']

X_train,X_temp,y_train,y_temp=train_test_split(X,y,test_size=0.2,random_state=42)
X_val,X_test,y_val,y_test=train_test_split(X_temp,y_temp,test_size=0.5,random_state=42)

# X_train, y_train (80%)
# X_val, y_val (10%)
# X_test, y_test (10%)

print("Data shapes:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

Data shapes:
X_train: (6000, 31)
y_train: (6000,)
X_val: (750, 31)
y_val: (750,)
X_test: (750, 31)
y_test: (750,)


## 方案二：二分法（训练集 + 测试集）⭐ 推荐

由于调参函数大多自带交叉验证，实际使用中只需要划分训练集和测试集。

In [4]:
from sklearn.model_selection import train_test_split
X = data.drop(['Credit Default'], axis=1)  # 特征，axis=1表示按列删除
y = data['Credit Default'] # 标签
# 按照8:2划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 80%训练集，20%测试集

## 导入评估工具

In [5]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # 用于评估分类器性能的指标
from sklearn.metrics import classification_report, confusion_matrix #用于生成分类报告和混淆矩阵
import warnings #用于忽略警告信息
warnings.filterwarnings("ignore") # 忽略所有警告信息

# 调参方法
## 三种主流方法对比
| 方法 | 原理 | 优点 | 缺点 | 适用场景 |
|------|------|------|------|----------|
| **网格搜索** | 穷举所有参数组合 | 能找到最优解 | 计算量大，维度灾难 | 参数空间小，计算资源充足 |
| **随机搜索** | 随机采样参数组合 | 效率高于网格搜索 | 可能错过最优解 | 参数空间大，中等计算资源 |
| **贝叶斯优化** | 基于概率模型智能搜索 | 高效，收敛快 | 实现复杂 | 参数空间大，计算资源有限 |

## 基线模型（Baseline）

在调参前，先建立基线模型：
- 使用**默认参数**训练模型
- 记录性能指标作为**对比基准**
- 后续调参效果以此为参照

## 详细说明

1️⃣ 网格搜索 (GridSearchCV)
- 需要定义参数的**固定列表**（param_grid）
- 尝试所有可能的参数组合
- ⚠️ 计算成本高，参数多时组合呈指数级增长

2️⃣ 随机搜索 (RandomizedSearchCV)
- 定义参数的**分布范围**
- 随机采样指定次数（如 50-100 次）
- ✅ 对于给定计算预算，通常比网格搜索更有效

3️⃣ 贝叶斯优化 (BayesSearchCV)
- 定义参数的**搜索空间**
- 根据先验结果建立概率模型（高斯过程）
- 智能选择下一个最有潜力的参数组合
- ✅ 通常用更少迭代达到更好效果

## 选择建议

```
计算资源充足 → 网格搜索
计算资源有限 → 贝叶斯优化
介于中间     → 随机搜索
```

---

# LightGBM调参
## 基准模型

In [6]:
import time
print('---1. 默认参数LightGBM---')
start_time = time.time()
lgb_model=lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train,y_train)
lgb_pred=lgb_model.predict(X_test)
end_time=time.time()

print(f'耗时：{(end_time-start_time):.4f}s')
print('LightGBM分类报告：')
print(classification_report(y_test,lgb_pred))
print('LightGBM混淆矩阵')
print(confusion_matrix(y_test,lgb_pred))

---1. 默认参数LightGBM---
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1672, number of negative: 4328
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001676 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2160
[LightGBM] [Info] Number of data points in the train set: 6000, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.278667 -> initscore=-0.951085
[LightGBM] [Info] Start training from score -0.951085
耗时：0.3605s
LightGBM分类报告：
              precision    recall  f1-score   support

           0       0.78      0.94      0.85      1059
           1       0.70      0.36      0.47       441

    accuracy                           0.77      1500
   macro avg       0.74      0.65      0.66      1500
weighted avg       0.75      0.77      0.74      1500

LightGBM混淆矩阵
[[992  67]
 [284 157]]


## 网格搜索
网格搜索会尝试所有参数组合，计算量较大但能找到局部最优解。

In [8]:
# --- 2. 网格搜索优化 LightGBM ---
print("\n--- 2. 网格搜索优化 LightGBM (训练集 -> 测试集) ---")
import time
from sklearn.model_selection import GridSearchCV

# 定义 LightGBM 的参数网格（针对分类任务，选核心可调参数）
param_grid = {
    'n_estimators': [50, 100, 200],  # 决策树数量（LightGBM 默认100，范围适度扩大）
    'learning_rate': [0.01, 0.05, 0.1],  # 学习率（LightGBM 核心参数，控制步长）
    'max_depth': [10, 20, 30],  # 树的最大深度（-1表示不限制，默认-1）
    'num_leaves': [1, 2, 4],  # 叶子节点数（LightGBM 特有参数，需小于 2^max_depth）
}

# 创建网格搜索对象（核心逻辑和随机森林一致）
grid_search_lgb = GridSearchCV(
    estimator=lgb.LGBMClassifier(
        random_state=42,  # 固定随机种子，保证可复现
        verbose=-1  # 训练时不输出冗余日志（0=默认，-1=静默）
    ),
    param_grid=param_grid,  # 上面定义的参数网格
    cv=5,  # 5折交叉验证（和随机森林保持一致）
    n_jobs=-1,  # 利用所有CPU核心并行计算（加速搜索）
    scoring='accuracy',  # 用准确率评分（和随机森林一致，方便对比）
    refit=True  # 搜索完成后，用最佳参数在全量训练集上重新训练模型
)

# 计时并执行网格搜索
start_time = time.time()
grid_search_lgb.fit(X_train, y_train)  # 在训练集上搜索最佳参数
end_time = time.time()

# 输出搜索结果
print(f"网格搜索耗时: {end_time - start_time:.4f} 秒")
print("最佳参数: ", grid_search_lgb.best_params_)  # 输出最优参数组合

# 使用最佳参数的模型进行预测
best_lgb_model = grid_search_lgb.best_estimator_  # 获取最优模型
best_lgb_pred = best_lgb_model.predict(X_test)  # 在测试集上预测

print("\n网格搜索优化后的 LightGBM 在测试集上的分类报告：")
print(classification_report(y_test, best_lgb_pred))
print("网格搜索优化后的 LightGBM 在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, best_lgb_pred))


--- 2. 网格搜索优化 LightGBM (训练集 -> 测试集) ---
网格搜索耗时: 9.4160 秒
最佳参数:  {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'num_leaves': 4}

网格搜索优化后的 LightGBM 在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.76      0.97      0.85      1059
           1       0.77      0.28      0.41       441

    accuracy                           0.76      1500
   macro avg       0.77      0.62      0.63      1500
weighted avg       0.77      0.76      0.72      1500

网格搜索优化后的 LightGBM 在测试集上的混淆矩阵：
[[1022   37]
 [ 316  125]]


## 随机搜索优化
随机搜索在参数空间中随机采样，通常比网格搜索更高效。

在复杂项目上随机优化比贝叶斯差很多，再简单场景比贝叶斯效率高，但是没必要



In [9]:
# --- 3. 随机搜索优化 LightGBM ---
print("\n--- 3. 随机搜索优化 LightGBM (训练集 -> 测试集) ---")
import time
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

# 定义参数分布（注意：随机搜索用「分布」而非固定列表，更灵活）
param_dist = {
    'n_estimators': np.random.randint(50, 200, size=10),  # 100-500间随机取10个整数
    'learning_rate': np.random.uniform(0.01, 0.2, size=10),  # 0.01-0.2间随机取10个浮点数
    'max_depth': [None, 10, 20, 30],
    'num_leaves': np.random.randint(31, 63, size=5),  # 31-63间随机取5个整数（避免过拟合）
}

# 创建随机搜索对象（API与GridSearchCV高度一致）
random_search_lgb = RandomizedSearchCV(
    estimator=lgb.LGBMClassifier(
        random_state=42,
        verbose=-1  # 静默训练，不输出日志
    ),
    param_distributions=param_dist,  # 传入参数分布（而非固定网格）
    n_iter=50,  # 随机采样50组参数（可调整，越大越可能找到最优解，但耗时更长）
    cv=5,  # 5折交叉验证（与网格搜索一致）
    n_jobs=-1,  # 并行计算
    scoring='accuracy',  # 准确率评分
    random_state=42,  # 固定随机种子，保证可复现
    refit=True  # 用最佳参数重新训练全量训练集
)

# 计时并执行随机搜索
start_time = time.time()
random_search_lgb.fit(X_train, y_train)
end_time = time.time()

# 输出结果
print(f"随机搜索耗时: {end_time - start_time:.4f} 秒")
print("最佳参数: ", random_search_lgb.best_params_)

# 最佳模型预测与评估
best_random_lgb = random_search_lgb.best_estimator_
best_random_pred = best_random_lgb.predict(X_test)

print("\n随机搜索优化后的 LightGBM 在测试集上的分类报告：")
print(classification_report(y_test, best_random_pred))
print("随机搜索优化后的 LightGBM 在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, best_random_pred))


--- 3. 随机搜索优化 LightGBM (训练集 -> 测试集) ---
随机搜索耗时: 58.5802 秒
最佳参数:  {'num_leaves': 31, 'n_estimators': 195, 'max_depth': 30, 'learning_rate': 0.027692831347809525}

随机搜索优化后的 LightGBM 在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.77      0.96      0.86      1059
           1       0.77      0.33      0.46       441

    accuracy                           0.77      1500
   macro avg       0.77      0.64      0.66      1500
weighted avg       0.77      0.77      0.74      1500

随机搜索优化后的 LightGBM 在测试集上的混淆矩阵：
[[1015   44]
 [ 295  146]]


## 贝叶斯优化

In [10]:
# --- 4. 贝叶斯优化 LightGBM ---
print("\n--- 4. 贝叶斯优化 LightGBM (训练集 -> 测试集) ---")
import time
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical  # 定义参数空间的类

# 定义贝叶斯优化的参数空间（用skopt的空间类，支持整数/浮点数/分类类型）
param_space = {
    'n_estimators': Integer(50, 200),  # 整数型：100-500
    'learning_rate': Real(0.01, 0.2, prior='log-uniform'),  # 浮点型：0.01-0.2（对数分布，更关注小学习率）
    'max_depth': Integer(10, 30),
    'num_leaves': Integer(31, 63),  # 整数型：31-255
}

# 创建贝叶斯搜索对象
bayes_search_lgb = BayesSearchCV(
    estimator=lgb.LGBMClassifier(
        random_state=42,
        verbose=-1
    ),
    search_spaces=param_space,  # 传入贝叶斯参数空间
    n_iter=30,  # 采样30组参数（比随机搜索少，但效果更优）
    cv=5,  # 5折交叉验证
    n_jobs=-1,
    scoring='accuracy',
    random_state=42,
    refit=True,
    n_points=5,  # 每次迭代采样5个参数点（默认5，可调整）
)

# 计时并执行贝叶斯优化
start_time = time.time()
bayes_search_lgb.fit(X_train, y_train)
end_time = time.time()

# 输出结果
print(f"贝叶斯搜索耗时: {end_time - start_time:.4f} 秒")
print("最佳参数: ", bayes_search_lgb.best_params_)

# 最佳模型预测与评估
best_bayes_lgb = bayes_search_lgb.best_estimator_
best_bayes_pred = best_bayes_lgb.predict(X_test)

print("\n贝叶斯优化后的 LightGBM 在测试集上的分类报告：")
print(classification_report(y_test, best_bayes_pred))
print("贝叶斯优化后的 LightGBM 在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, best_bayes_pred))


--- 4. 贝叶斯优化 LightGBM (训练集 -> 测试集) ---
贝叶斯搜索耗时: 38.6247 秒
最佳参数:  OrderedDict([('learning_rate', 0.01), ('max_depth', 16), ('n_estimators', 200), ('num_leaves', 39)])

贝叶斯优化后的 LightGBM 在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.76      0.97      0.85      1059
           1       0.79      0.28      0.41       441

    accuracy                           0.77      1500
   macro avg       0.78      0.62      0.63      1500
weighted avg       0.77      0.77      0.72      1500

贝叶斯优化后的 LightGBM 在测试集上的混淆矩阵：
[[1026   33]
 [ 318  123]]
